In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import glob
import pandas as pd

import nltk
from nltk import word_tokenize
nltk.download('punkt')
from nltk.tag import StanfordNERTagger

import string
import json
import re
import os
import csv

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from nltk.tag import StanfordNERTagger

#Check that paths are set properly for local host
jar = 'drive/My Drive/Colab Notebooks/stanford-ner.jar'
model = 'drive/My Drive/Colab Notebooks/english.all.3class.distsim.crf.ser.gz'

ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

/usr/local/lib/python3.6/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [0]:
!apt-get update

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [83.2 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [266 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [575 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1

In [0]:
import os    
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     
  !java -version      
install_java()

#Taken from: https://stackoverflow.com/questions/51287258/how-can-i-use-java-in-google-colab

openjdk version "1.8.0_191"
OpenJDK Runtime Environment (build 1.8.0_191-8u191-b12-0ubuntu0.18.04.1-b12)
OpenJDK 64-Bit Server VM (build 25.191-b12, mixed mode)


In [0]:
'''
Function "load" takes a .csv file as the parameter. It reads the .csv file and returns 
the file as a list of lists.
'''

def load(text):
    lines = []
    with open(text) as f:
        reader = csv.reader(f)
        for row in reader:
            lines.append(row)
    return lines

In [0]:
#Load csv
txt_files = load('drive/My Drive/Colab Notebooks/filtered_data_sub.csv') 

In [0]:
# [x][0] gives author
# [x][1] gives body

In [0]:
#Create list of unique authors
author_names = []
for i in range(len(txt_files)):
    temp_string = txt_files[i][0]
    author_names.append(temp_string)

In [0]:
#load comments
work_loaded = []
for i in range(len(txt_files)):
    temp_string = txt_files[i][1]
    work_loaded.append(temp_string)

In [0]:
#replace urls in comments
work =list(map(lambda x: re.sub(r"http\S+", "<url>",x ), work_loaded))

In [0]:
#Replace digits with '0'
digits = []
for word in work:
    temp_string = "".join(word)
    replaced_string = re.sub('\d', '0', temp_string)
    digits.append(replaced_string)

In [0]:
#Tokenize text
work_tokenized = []
for word in digits:
    work_tokenized.append(word_tokenize(word))

In [0]:
#Replace tokens tagged as PERSON, ORGANIZATION, or LOCATION with '#'
final_tokens = []
for i in range((len(work_tokenized))):
            tag_tokens = ner_tagger.tag(work_tokenized[i])
            tag_df = pd.DataFrame(tag_tokens, columns=['token', 'label'])
            tag_df.loc[tag_df.label == "PERSON", 'token'] = "#"
            tag_df.loc[tag_df.label == "ORGANIZATION", 'token'] = "#"
            tag_df.loc[tag_df.label == "LOCATION", 'token'] = "#"
            new_tokens = list(tag_df['token'])
            final_tokens.append(new_tokens)

In [0]:
#Create dictionary for authors with their corresponding tokens
works = {'author': author_names,
         'work': final_tokens}
author_works_df = pd.DataFrame.from_dict(works)
author_works_dict = {x: y["work"].tolist() for x,y in author_works_df.groupby("author")}

In [0]:
#Write author/tokens dictionary to .txt file
with open('authors_tokens_dict.txt', 'w') as file:
     file.write(json.dumps(author_works_dict))

In [0]:
#Write author/tokens dictionary to .csv file
with open('authors_tokens_dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in author_works_dict.items():
        writer.writerow([key, value])

In [0]:
author_works_df.to_csv('author_tokens_df.csv', encoding='utf-8', index=False)

In [0]:
'''
csv.field_size_limit(100000000)
with open('authors_tokens_dict.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)
'''

"\ncsv.field_size_limit(100000000)\nwith open('authors_tokens.csv', 'r') as csv_file:\n    reader = csv.reader(csv_file)\n    mydict = dict(reader)\n"